Import dependencies 

In [1]:
import time
from selenium import webdriver
from selenium.webdriver import Chrome 
from selenium.webdriver.common.by import By 
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import re
import pandas as pd
import numpy as np

import os
import googleapiclient.discovery

from wordcloud import WordCloud
import matplotlib.pyplot as plt

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup



Youtube Comments Scrapper

In [2]:
def scrapper_comments_youtube(id):
    # Disable OAuthlib's HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    api_service_name = "youtube"
    api_version = "v3"
    DEVELOPER_KEY = process.env.DEVELOPER_KEY

    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, developerKey = DEVELOPER_KEY)

    request = youtube.commentThreads().list(
        part="id,snippet",
        maxResults=10000,
        order="relevance",
        videoId= id
    )
    response = request.execute()

    authorname = []
    comments = []
    for i in range(len(response["items"])):
        authorname.append(response["items"][i]["snippet"]["topLevelComment"]["snippet"]["authorDisplayName"])
        comments.append(response["items"][i]["snippet"]["topLevelComment"]["snippet"]["textOriginal"])
        df = pd.DataFrame(comments, index = authorname,columns=["Comments"])
    return df

Sentiment Score

In [3]:
def sentiment_score(review):
    tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
    model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1 
    

In [4]:
def sentiment_score_new_column(dataframe):
    dataframe['sentiment'] = dataframe['Comments'].apply(lambda x:sentiment_score(x[:512]))

WordCloud Function

In [5]:
def wordcloud(dataframe):
    wc = WordCloud()
    wc.generate(str(dataframe['Comments']))
    plt.imshow(wc,interpolation='bilinear')
    plt.show()

TEST

In [6]:
result =scrapper_comments_youtube('qU9mHegkTc4')

NameError: name 'process' is not defined

In [ ]:
sentiment_score_new_column(result)

In [ ]:
wordcloud(result)

In [ ]:
result

Visualize the results 

In [ ]:
df

Write Results into a csv 

In [23]:
result.to_csv('df.csv')